In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.cm as cm
import matplotlib as mpl
import xesmf as xe
from workflow.scripts.utils import regrid_global
import numpy as np
from pyclim_noresm.general_util_funcs import global_avg
import scipy.stats as stats
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import curve_fit
from numpy.linalg import lstsq
import scipy

In [3]:
def read_data(paths, tag='experiment_id'):
    dsets = []
    for p in paths:
        ds = xr.open_dataset(p)
        ds = ds.cf.add_bounds(['lon','lat'])
        grid_params = snakemake.config['regrid_params']
        method=grid_params.get('method','conservative')

        dxdy = grid_params['dxdy']
        ds = regrid_global(ds, lon=dxdy[0], lat=dxdy[1], method=method)
        da = ds[ds.variable_id]
#         return da
        da = da.rename(f'{ds.variable_id}_{ds.attrs[tag]}')
        da.attrs['source_id'] = ds.source_id 
        da.attrs['experiment_id'] = ds.experiment_id
        da.attrs['variable_id'] = ds.variable_id
        da = da.assign_coords(time=np.arange(0, len(da.time)))
#         da = da.rename(year='time')
        dsets.append(da)
    out_da = xr.merge(dsets)
    return out_da


In [4]:
rad_noresm = read_data(snakemake.input.noresm_rad)

load_nor = read_data(snakemake.input.noresm_load)

clt_nor = read_data(snakemake.input.noresm_clt)

In [5]:
exp_noresm = list({rad_noresm[d].experiment_id for d in rad_noresm.data_vars})
variables = list({load_nor[v].variable_id for v in load_nor.data_vars})

In [37]:
def calc_toa_imbalance(ds_rad, experiments,ds_load=None,tag='cs', subtract_mean_dTOA=True, sw=False):
    toa_imbalance = []
    loads = []
    for exp in experiments:
        if sw ==True:
            temp_da = (np.abs(ds_rad[f'rsdt_{exp}'].dropna(dim='time')) 
#                    - np.abs(ds_rad[f'rlut{tag}_{exp}'].dropna(dim='time')) 
                   - np.abs(ds_rad[f'rsut{tag}_{exp}'].dropna(dim='time')))
        
        else:
            temp_da = (np.abs(ds_rad[f'rsdt_{exp}'].dropna(dim='time')) 
                   - np.abs(ds_rad[f'rlut{tag}_{exp}'].dropna(dim='time')) 
                   - np.abs(ds_rad[f'rsut{tag}_{exp}'].dropna(dim='time')))
        
        if ds_load:
            load_da = ds_load[f'mmrdust_{exp}'].dropna(dim='time')*ds_load[f'airmass_{exp}'].dropna(dim='time')
            load_da = load_da.sum(dim='lev')
    #         return load_da
            load_da = global_avg(load_da)
            load_da = load_da -load_da.mean()
            load = load_da.to_dataset(name=f'loaddust_{exp}')
            loads.append(load)
        temp_da = global_avg(temp_da)
        if subtract_mean_dTOA:
            temp_da = temp_da - temp_da.mean()
        temp_rad = temp_da.to_dataset(name=f'dTOA_{exp}')
        
        toa_imbalance.append(temp_rad)
    load_im=xr.merge(toa_imbalance+loads)
#     if subtract_mean == True:
#         load_im=load_im-load_im.mean()
    
    return load_im

In [35]:
def calc_loads(ds, variable, experiment):
    dsets = []
    for v in variables:
        for exp in experiment: 
            if v == 'airmass':
                continue
            else:
                dsmmr = ds[f'{v}_{exp}'].dropna(dim='time')
                dsair = ds[f'airmass_{exp}'].dropna(dim='time')
#                 print(dsair.shape)
                load = dsmmr*dsair
                load = load.sum(dim='lev')
                
                load = global_avg(load)
                load = load-load.mean()
                load = load.to_dataset(name=f'load{v[3:]}_{exp}')
                dsets.append(load)
    ds = xr.merge(dsets)
    return ds

In [36]:
# ec_load_im = calc_toa_imbalance(rad_ec,exp_ec,ds_load=load_ec)
nor_im = calc_toa_imbalance(rad_noresm, exp_noresm)
nor_im_sw = calc_toa_imbalance(rad_noresm, exp_noresm, sw=True)

In [8]:
nor_clt = global_avg(clt_nor).to_dataframe()

In [24]:
clt = nor_clt.replace(0.0,np.nan).values.ravel()
clt = clt[~np.isnan(clt)]

In [38]:
nor_load = calc_loads(load_nor, variables, exp_noresm)

In [39]:
def stack_data(ds_im,ds_load, experiments, variable):
    df = ds_im.to_dataframe()
    dfl = ds_load.to_dataframe()
    dfs = []
    
    for exp in experiments:
        tdf = df[f'dTOA_{exp}'].to_frame()
        tdfl = dfl[f'{variable}_{exp}']
        tdf[f'{variable}_{exp}'] = tdfl
        

        tdf = tdf.dropna()
#         tdf=tdf[(np.abs(stats.zscore(tdf)) < 3).all(axis=1)]
#         return tdf, tdfl
#         return tdf
        tdf = tdf.rename(columns={f'{variable}_{exp}':variable,f'dTOA_{exp}':'dTOA'})
        dfs.append(tdf)
#     ts_load_ano = [df[[f'loaddust_{exp}']] for exp in experiments]
#     ts_dTOA_ano = [df[f'dTOA_{exp}'] for exp in experiments]
    df = pd.concat(dfs).reset_index()
#     ts_dTOA = pd.concat(ts_dTOA_ano).reset_index()
#     return tdf
    df = df.drop(columns='time')
    df=df[(np.abs(stats.zscore(df['dTOA'])) < 3)]
#      = ts_load.drop(columns='time')
    return df
        

In [40]:
nor_loadss_dTOA = stack_data(nor_im, nor_load,exp_noresm,'loadss')
nor_loadsoa_dTOA = stack_data(nor_im, nor_load,exp_noresm,'loadsoa')
nor_loadso4_dTOA = stack_data(nor_im, nor_load,exp_noresm,'loadso4')

nor_loadbc_dTOA = stack_data(nor_im, nor_load,exp_noresm,'loadbc')
nor_loaddust_dTOA = stack_data(nor_im, nor_load,exp_noresm,'loaddust')


In [30]:
nor_clt_dTOA = pd.DataFrame(clt, columns=['clt'])
nor_clt_dTOA = nor_clt_dTOA-nor_clt_dTOA.mean()
nor_clt_dTOA['dTOA'] = load

In [41]:
nor_clt_dTOA['dTOA'] = nor_loadbc_dTOA['dTOA']

In [12]:

def make_fit_numpy(df,variable,regularized = True):
    a,c = np.polyfit(df[variable], df['dTOA'],1)
#     model = sm.OLS(df['dTOA'],X, missing='drop')
    return a,c,a

In [43]:

def make_fit_scipy(df,variable,regularized = True):
    x=df[variable]
    y= df['dTOA']
    res=scipy.stats.linregress(x, y)
#  'dTOA'],bounds=(0, [3., 1., 0.5]))   model = sm.OLS(df['dTOA'],X, missing='drop')
    return res, res.slope, res.intercept

In [14]:
def make_fit(df,variable,regularized = True):
    X = sm.add_constant(df[variable])
    model = sm.OLS(df['dTOA'],X, missing='drop')
    if regularized:
        result = model.fit_regularized(L1_wt=1)
    else:
        result = model.fit()
    return result, result.params.values[1], result.params.values[0]

In [46]:
def plot_fit():
    nor_loadss_dTOA = stack_data(nor_im, nor_load,exp_noresm,'loadss')
    nor_loadsoa_dTOA = stack_data(nor_im, nor_load,exp_noresm,'loadsoa')
    nor_loadso4_dTOA = stack_data(nor_im, nor_load,exp_noresm,'loadso4')

    nor_loadbc_dTOA = stack_data(nor_im, nor_load,exp_noresm,'loadbc')
    nor_loaddust_dTOA = stack_data(nor_im, nor_load,exp_noresm,'loaddust')
    dfs = [nor_loadss_dTOA, nor_loadso4_dTOA,nor_loaddust_dTOA,nor_clt_dTOA]
    variable = ['loadss', 'loadso4','loaddust']
    fig,ax = plt.subplots(figsize=(14,4.5), ncols=len(variable))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    xn = np.linspace(-1,1,100)
    xlims=[(-8e-7,8e-7),(-1e-7, 1e-7),(-2e-6, 2e-6),(1,-1)]
    for axi, v, df,xlim in zip(ax, variable, dfs, xlims):
        res_nor, c_nor,x_nor = make_fit_scipy(df, v,False)
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        axi.scatter(df[v],df['dTOA'], s=5)
        yn = np.polyval([c_nor, x_nor], xn)
        axi.plot(xn,yn, linewidth=3, color='grey')
        axi.set_ylim(-0.5,0.5)
        axi.set_xlim(xlim)
        axi.set_title(v)
        axi.text(0.17, 0.08, f'$R^2 = $ {res_nor.rvalue**2:.2f} \n nobs = {len(df)}', 
               va='center', ha='center', transform=axi.transAxes)
        axi.text(0.5,0.92, f'dTOA={c_nor:.2E} $* \;C_{{v}}$',
              va='center', ha='center', transform=axi.transAxes,fontsize=14, bbox=props)
        axi.set_xlabel('Load anomaly \n [kg m-2]')
    
    ax[0].set_ylabel('TOA imbalance [W m-2]')
plot_fit()
plt.savefig(snakemake.output.toa_load_png, bbox_inches='tight', dpi=144)

In [40]:
def plot_fit_sw():
    nor_loadss_dTOA = stack_data(nor_im_sw, nor_load,exp_noresm,'loadss')
    nor_loadsoa_dTOA = stack_data(nor_im_sw, nor_load,exp_noresm,'loadsoa')
    nor_loadso4_dTOA = stack_data(nor_im_sw, nor_load,exp_noresm,'loadso4')

    nor_loadbc_dTOA = stack_data(nor_im_sw, nor_load,exp_noresm,'loadbc')
    nor_loaddust_dTOA = stack_data(nor_im_sw, nor_load,exp_noresm,'loaddust')
    dfs = [nor_loadss_dTOA, nor_loadso4_dTOA,nor_loaddust_dTOA]
    variable = ['loadss', 'loadso4','loaddust']
    fig,ax = plt.subplots(figsize=(14,4.5), ncols=len(variable))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    xn = np.linspace(-10e-4,10e-4,100)
    xlims=[(-8e-7,8e-7),(-1e-7, 1e-7),(-2e-6, 2e-6),]
    for axi, v, df,xlim in zip(ax, variable, dfs, xlims):
        res_nor, c_nor,x_nor = make_fit_scipy(df, v,False)
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        axi.scatter(df[v],df['dTOA'], s=5)
        yn = np.polyval([c_nor, x_nor], xn)
        axi.plot(xn,yn, linewidth=3, color='grey')
        axi.set_ylim(-0.5,0.5)
        axi.set_xlim(xlim)
        axi.set_title(v)
        axi.text(0.17, 0.08, f'$R^2 = $ {res_nor.rvalue**2:.2f} \n nobs = {len(df)}', 
               va='center', ha='center', transform=axi.transAxes)
        axi.text(0.5,0.92, f'dTOA={c_nor:.2E} $* \;C_{{v}}$',
              va='center', ha='center', transform=axi.transAxes,fontsize=14, bbox=props)
        axi.set_xlabel('Load anomaly \n [kg m-2]')
    
    ax[0].set_ylabel('TOA imbalance [W m-2]')
plot_fit_sw()
plt.savefig(snakemake.output.toa_load_sw_png, bbox_inches='tight', dpi=144)

In [118]:
val=False
if val:
    histSST = read_data(snakemake.input.histSST_rad, tag='source_id')

    mmr_mass_hist = read_data(snakemake.input.histSST_load, tag='source_id')

    models = list({histSST[d].attrs['source_id'] for d in histSST.data_vars})

    def calc_load(mmr_air, models):
        dsets = []
        for model in models:
            ds_load = mmr_air[f'mmrdust_{model}']*mmr_air[f'airmass_{model}']
            ds_load = ds_load.sum(dim='lev')
            ds_load = global_avg(ds_load)
            ds_load.attrs['units'] = 'kg m-2'
            ds_load.attrs['long_name'] = 'Load of Dust'
            ds_load = ds_load.to_dataset(name=f'loaddust_{model}')
            dsets.append(ds_load)
        ds = xr.merge(dsets)
        return ds



    histSST_imbalance_cs = calc_toa_imbalance(histSST,models,tag='cs', subtract_mean_dTOA=False)

    histSST_imbalance_cs

    res_nor, c_nor,x_nor = make_fit(nor_load_dTOA, False)
    res_ec, c_ec,x_ec = make_fit(ec_load_dTOA, False)
    res_mpi, c_mpi, x_mpi = make_fit(mpi_load_dTOA, False)

    loads_histSST = calc_load(mmr_mass_hist, models)

    def create_validation_data(loading, dTOA, t_slice=None):
        if t_slice:
            loading = loading.isel(time=t_slice)
            dTOA = dTOA.isel(time=t_slice)
        else:
            loading = loading.isel(time=slice(0,30))
            dTOA = dTOA.sel(time=slice(0,30))

        loading = loading - loading.mean()
        loading = loading.rename('loaddust')
    #     dTOA = dTOA - dTOA.mean()
        dTOA = dTOA.to_dataset(name='dTOA')
        dfTOA = dTOA.to_pandas()
        dfTOA['loaddust'] = loading.to_pandas()
    #     dfloading = 
        return dfTOA

    nor_val = create_validation_data(loads_histSST['loaddust_NorESM2-LM'], histSST_imbalance_cs['dTOA_NorESM2-LM'])
    nor_val['predicted'] = res_nor.predict(sm.add_constant(nor_val['loaddust']))
    mpi_val = create_validation_data(loads_histSST['loaddust_MPI-ESM-1-2-HAM'], 
                                     histSST_imbalance_cs['dTOA_MPI-ESM-1-2-HAM'], slice(-40,-10))
    mpi_val['predicted'] = res_mpi.predict(sm.add_constant(mpi_val['loaddust']))

    ec_val = create_validation_data(loads_histSST['loaddust_EC-Earth3-AerChem'], 
                                     histSST_imbalance_cs['dTOA_EC-Earth3-AerChem'])
    ec_val['predicted'] = res_mpi.predict(sm.add_constant(ec_val['loaddust']))

    dhTOA_mpi = histSST_imbalance_cs['dTOA_MPI-ESM-1-2-HAM'].isel(time=slice(0,30)) - histSST_imbalance_cs['dTOA_MPI-ESM-1-2-HAM'].isel(time=slice(0,30)).mean()

In [68]:
if val:
    ax = plt.gca()
    mpi_load_im['loaddust_piClim-aer'].plot(ax=ax)
    mpi_load_im['loaddust_piClim-2xdust'].plot(ax=ax)
    mpi_load_im['loaddust_piClim-control'].plot(ax=ax)
    mpi_val['loaddust'].plot(ax=ax)

In [100]:
if val:
    ax = plt.gca()

    ax.plot(res_mpi.predict(sm.add_constant(mpi_load_im['loaddust_piClim-aer'])))
    ax.plot(res_mpi.predict(sm.add_constant(mpi_load_im['loaddust_piClim-2xdust'])))
    ax.plot(res_mpi.predict(sm.add_constant(mpi_load_im['loaddust_piClim-control'])))
    (mpi_val['predicted']).plot(ax=ax)

In [108]:
if val:
    ax=plt.gca()
    # mpi_no_dust.plot(ax=ax,marker='o', label='no dust')
    ((mpi_val['dTOA']-mpi_val['dTOA'].mean())-mpi_val['predicted']).plot(ax=ax, marker='s', label='dTOA histSST')
    ((mpi_val['dTOA']-mpi_val['dTOA'].mean())).plot(ax=ax, marker='s', label='dTOA histSST')
    # ax.set_ylim(-0.6, 0.6)
    ax.legend()

In [120]:
if val:
    mpi_no_dust_hist = (dhTOA_mpi-
                        res_mpi.predict(sm.add_constant(mpi_val['loaddust'])))

    ax = plt.gca()
    dhTOA_mpi.plot(ax=ax)
    mpi_no_dust_hist.plot(ax=ax)

In [119]:
if val:
    ax = plt.gca()
    # mpi_val['predicted'].plot(ax=ax)
    (mpi_val['dTOA']-mpi_val['dTOA'].mean()).plot(ax=ax)
    (mpi_val['dTOA']-(mpi_val['predicted']+mpi_val['dTOA'].mean())).plot(ax=ax)